In [4]:
#convert python txt <p1> <p2> file to json file format
import json
f = open("mapping_db_wk_clean.txt",'r',encoding='utf-8').read()
json_dict = {"values" : []}
for line in f.split("\n"):
    values = line.split(" ")
    if len(values) == 2 : db,wk = values
    json_dict["values"].append({"db" : db, "wk": wk})
with open("mapping_db_wk.json", "w") as outfile:
    json.dump(json_dict, outfile)

In [1]:
import json
#compare sameAs prop with clean db prop
with open('mapping_db_wk.json', 'r') as openfile:
    mapping_dict = json.load(openfile)

#load mapping into a dict to get constant acces time
db_sameAs_dict = {}
for i in mapping_dict["values"]:
    db_sameAs_dict[i["db"]] = i["wk"]

with open("../data/property_support.json") as json_file:
        property_dictionnary = json.load(json_file)
            


In [3]:
#filter prop count dictionary with sameAs dictionary
filtered_prop = {}
for i in db_sameAs_dict.keys():
    val = property_dictionnary.get(i)
    if val: filtered_prop[i] = val

#sort the dict
prop = dict(sorted(filtered_prop.items(), key=lambda item: item[1], reverse=True))
#keep only prop with support > 1000
prop = dict(filter(lambda x,: True if x[1] > 1000 else False, prop.items()))



In [7]:
db_sameAs_dict['<http://dbpedia.org/ontology/deathDate>']

'<http://www.wikidata.org/prop/P570>'

In [13]:
for i in prop.keys():
        print(f"{db_sameAs_dict[i]} sameAs {i} : {prop[i]}")


<http://www.wikidata.org/prop/P11143> sameAs <http://dbpedia.org/ontology/wikiPageID> : 1406259
<http://www.wikidata.org/prop/P2451> sameAs <http://dbpedia.org/ontology/birthDate> : 471625
<http://www.wikidata.org/prop/P570> sameAs <http://dbpedia.org/ontology/deathDate> : 170365
<http://www.wikidata.org/prop/P2907> sameAs <http://dbpedia.org/ontology/utcOffset> : 163232
<http://www.wikidata.org/prop/P6499> sameAs <http://dbpedia.org/ontology/populationTotal> : 154981
<http://www.wikidata.org/prop/P8497> sameAs <http://dbpedia.org/ontology/Person/height> : 111006
<http://www.wikidata.org/prop/P8483> sameAs <http://dbpedia.org/ontology/postalCode> : 106248
<http://www.wikidata.org/prop/P5286> sameAs <http://dbpedia.org/ontology/runtime> : 97818
<http://www.wikidata.org/prop/P1477> sameAs <http://dbpedia.org/ontology/originalName> : 70857
<http://www.wikidata.org/prop/P10708> sameAs <http://dbpedia.org/ontology/areaCode> : 66970
<http://www.wikidata.org/prop/P10673> sameAs <http://dbpedi

### Test values 

In [1]:
#import 
import time
from SPARQLWrapper import SPARQLWrapper, JSON
import re

In [2]:
#const
DBPEDIA = "http://dbpedia.org/sparql"
WIKIDATA = "https://query.wikidata.org/sparql"

In [3]:
#isbn - wdt:P957

In [4]:
def sparql_query(query : str, endpoint : str ) -> dict:
    sparql = SPARQLWrapper(
        endpoint
    )
    sparql.setReturnFormat(JSON)
    sparql.setQuery(query)

    try:
        ret = sparql.queryAndConvert()
        return ret
    except Exception as e:
        print(e)
        raise Exception("Error in query")

In [5]:
def query_value_sample(prop : str) -> str:
    query = """
                select ?e ?p ?v where {
                    values ?p {"""+prop+"""}.
                    ?e ?p ?v
                } limit 100
            """
    query = re.sub(r"\n|'","",query)
    return query


In [7]:
#find prop to query
db_prop = "<http://dbpedia.org/ontology/releaseDate>"
wk_prop = "<http://www.wikidata.org/prop/statement/P10673>"
result_db = sparql_query(query_value_sample(db_prop),DBPEDIA)
result_wk = sparql_query(query_value_sample(wk_prop),WIKIDATA)

In [14]:
date_i = []
for i,j in zip(result_db["results"]["bindings"],result_wk["results"]["bindings"]):
    #print(i)
    #print(i['v']['value'],j['v']['value'])
    date_i.append(i['v']['value'])

In [15]:
date_i

['2006-10-28',
 '2005-11-15',
 '1936-11-07',
 '1955-09-08',
 '1936-12-07',
 '1937-04-12',
 '1974-12-02',
 '1972-06-30',
 '1955-07-06',
 '2006-09-09',
 '2007-11-06',
 '2007-11-13',
 '2009-09-29',
 '2014-03-25',
 '2001-12-04',
 '2004-09-06',
 '2004-11-10',
 '2004-11-23',
 '2004-12-09',
 '2005-09-27',
 '2011-09-27',
 '2000-09-12',
 '1999-09-14',
 '2003-11-11',
 '2004-02-05',
 '2006-04-04',
 '2005-11-16',
 '2005-11-24',
 '2008-09-23',
 '2010-10-19',
 '2005-09-13',
 '2005-09-14',
 '2006-03-07',
 '2009-09-08',
 '2011-11-01',
 '2003-06-03',
 '1969-02-10',
 '2017-04-28',
 '2022-10-24',
 '2004-05-10',
 '2018-03-14',
 '2002-04-11',
 '2013-03-09']

In [18]:
from datetime import datetime

dates = [
    '2006-10-28', '2005-11-15', '1936-11-07', '1955-09-08', '1936-12-07',
    '1937-04-12', '1974-12-02', '1972-06-30', '1955-07-06', '2006-09-09',
    '2007-11-06', '2007-11-13', '2009-09-29', '2014-03-25', '2001-12-04',
    '2004-09-06', '2004-11-10', '2004-11-23', '2004-12-09', '2005-09-27',
    '2011-09-27', '2000-09-12', '1999-09-14', '2003-11-11', '2004-02-05',
    '2006-04-04', '2005-11-16', '2005-11-24', '2008-09-23', '2010-10-19',
    '2005-09-13', '2005-09-14', '2006-03-07', '2009-09-08', '2011-11-01',
    '2003-06-03', '1969-02-10', '2017-04-28', '2022-10-24', '2004-05-10',
    '2018-03-14', '2002-04-11', '2013-03-09'
]

american_dates = []
european_dates = []

for date_str in dates:
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    american_date = date_obj.strftime('%m/%d/%Y')
    american_dates.append(american_date)
    european_date = date_obj.strftime('%d/%m/%Y')
    european_dates.append(european_date)

# Print the converted dates
for a,e in zip(american_dates,european_dates):
    print(a+" - "+e)


10/28/2006 - 28/10/2006
11/15/2005 - 15/11/2005
11/07/1936 - 07/11/1936
09/08/1955 - 08/09/1955
12/07/1936 - 07/12/1936
04/12/1937 - 12/04/1937
12/02/1974 - 02/12/1974
06/30/1972 - 30/06/1972
07/06/1955 - 06/07/1955
09/09/2006 - 09/09/2006
11/06/2007 - 06/11/2007
11/13/2007 - 13/11/2007
09/29/2009 - 29/09/2009
03/25/2014 - 25/03/2014
12/04/2001 - 04/12/2001
09/06/2004 - 06/09/2004
11/10/2004 - 10/11/2004
11/23/2004 - 23/11/2004
12/09/2004 - 09/12/2004
09/27/2005 - 27/09/2005
09/27/2011 - 27/09/2011
09/12/2000 - 12/09/2000
09/14/1999 - 14/09/1999
11/11/2003 - 11/11/2003
02/05/2004 - 05/02/2004
04/04/2006 - 04/04/2006
11/16/2005 - 16/11/2005
11/24/2005 - 24/11/2005
09/23/2008 - 23/09/2008
10/19/2010 - 19/10/2010
09/13/2005 - 13/09/2005
09/14/2005 - 14/09/2005
03/07/2006 - 07/03/2006
09/08/2009 - 08/09/2009
11/01/2011 - 01/11/2011
06/03/2003 - 03/06/2003
02/10/1969 - 10/02/1969
04/28/2017 - 28/04/2017
10/24/2022 - 24/10/2022
05/10/2004 - 10/05/2004
03/14/2018 - 14/03/2018
04/11/2002 - 11/